# Import and clean large GNPS MS/MS dataset

## Using matchms to harmonize, clean, and complete metadata
Here we run matchms filters and additional custom filters to do extensive cleaning of a large spectral dataset (>210,000 spectra).

This notebook will expect that you have ``spec2vec`` installed via conda, which (currently) automatically include ``matchms`` and ``rdkit`` as well.
You can do this by creating an environment in anaconda and installing matchms via conda:

``conda create --name spec2vec_analysis python=3.8
conda activate spec2vec_analysis
conda install --channel nlesc --channel bioconda --channel conda-forge spec2vec``

In [1]:
import os
import numpy as np
from matchms.importing import load_from_mgf
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

ROOT = os.path.dirname(os.getcwd())
path_data = os.path.join(os.path.dirname(ROOT), "Data")  # add your local data folder here

## Load spectra raw data from GNPS
- Retrieved on 25/01/2021 from https://gnps-external.ucsd.edu/gnpslibrary

In [2]:
filename = os.path.join(path_data, 'GNPS_all', 'ALL_GNPS.mgf')
spectrums = list(load_from_mgf(filename))

print("number of spectra:", len(spectrums))

number of spectra: 210407


In [3]:
import pickle
pickle.dump(spectrums, 
            open(os.path.join(path_data, 'GNPS_all', 'ALL_GNPS.pickle'), "wb"))

In [3]:
import pickle
outfile = os.path.join(path_data, 'GNPS_all', 'ALL_GNPS.pickle')
with open(outfile, 'rb') as file:
    spectrums = pickle.load(file)

In [40]:
def count_annotations(spectra):
    inchi_lst = []
    smiles_lst = []
    inchikey_lst = []
    for i, spec in enumerate(spectra):
        inchi_lst.append(spec.get("inchi"))
        smiles_lst.append(spec.get("smiles"))
        inchikey = spec.get("inchikey")
        if inchikey is None:
            inchikey = spec.get("inchikey_inchi")
        inchikey_lst.append(inchikey)

    inchi_count = sum([1 for x in inchi_lst if x])
    smiles_count = sum([1 for x in smiles_lst if x])
    inchikey_count = sum([1 for x in inchikey_lst if x])
    print("Inchis:", inchi_count, "--", len(set(inchi_lst)), "unique")
    print("Smiles:", smiles_count, "--", len(set(smiles_lst)), "unique")
    print("Inchikeys:", inchikey_count, "--", 
          len(set([x[:14] for x in inchikey_lst if x])), "unique (first 14 characters)")

In [5]:
count_annotations(spectrums) 

Inchis: 207243 -- 18697 unique
Smiles: 208516 -- 22996 unique
Inchikeys: 0 -- 0 unique (first 14 characters)


## Run basic matchs filters

In [1]:
from matchms.filtering import default_filters
from matchms.filtering import add_parent_mass, derive_adduct_from_name

def apply_filters(s):
    s = default_filters(s)
    s = derive_adduct_from_name(s)
    s = add_parent_mass(s, estimate_from_adduct=True)
    return s

spectrums = [apply_filters(s) for s in spectrums]

NameError: name 'spectrums' is not defined

In [13]:
formulas = []
name_to_formulas = []
for spec in spectrums:
    if spec.get("formula"):
        formulas.append(spec.get("formula"))
        name_to_formulas.append(spec.get("compound_name") + "---" + spec.get("formula"))

In [15]:
name_to_formulas[:1000]

['Microcystin---LR',
 'Coproporphyrin---III',
 'Coproporphyrin I or---III',
 'Phosphatidylethanolamine (22:0/16:1) Abbr:---BPoPE',
 'Phosphatidylethanolamine GOPE or---ALPE',
 'Phosphatidylethanolamine (20:1/16:1) Abbr:---GPoPE',
 'Phosphatidylethanolamine (20:1/16:1) Abbr:---GPoPE',
 'Phosphatidylethanolamine (12:0/18:3) Abbr:---LaLnPE',
 'Phosphatidylethanolamine (16:0/18:2) Abbr:---PLPE',
 'Phosphatidylethanolamine (16:0/18:2) Abbr:---PLPE',
 'Phosphatidylethanolamine (16:1/18:3) Abbr:---PoLnPE',
 'Phosphatidylethanolamine (16:1/18:2) Abbr:---PoLPE',
 'Phosphatidylethanolamine PoLPE or---PLnPE',
 'Phosphatidylethanolamine PoOPE or---PLPE',
 'Phosphatidylethanolamine POPE or---PoSPE',
 'Phosphatidylethanolamine (16:1/18:0) Abbr:---PoSPE',
 'Phosphatidylethanolamine (16:0/16:1) Abbr: PPoPE or (14:0/18:1) Abbr:---MOPE',
 'Phosphatidylethanolamine (18:0/18:1) Abbr:---SOPE',
 'Protoporphyrin---IX',
 '---GE37468',
 '---GE37468',
 'Putative match to B.subtilis---SKF',
 'Putative match to B

In [16]:
len(formulas)

57036

In [17]:
len(list(set(formulas)))

1748

In [18]:
list(set(formulas))

['C43H74O13P1',
 'C20H41N1O7P1',
 'OXAPROZIN',
 'C45H89N1O7P1',
 'HOMOSERINE',
 'C21H43N1O7P1',
 'C50H91N1O8P1',
 'C48H89N1O7P1',
 'VALSARTAN',
 'NITRATE',
 'C39H80N2O6P1',
 'SACCHARATE',
 'SULFOXIMINE',
 'C59H104N1O6',
 'C14H30N1O2',
 'B27A12',
 'C48H94N1O8',
 'MMV676270',
 'C57H114N1O6',
 'C2H5S',
 'IC202C',
 'C51H92N1O6',
 'QUINATE',
 'NIN',
 'C45H81N1O8P1',
 'MMV676269',
 'C43H79N1O7P1',
 'C29H54O12P1',
 'C65H102N1O6',
 'GLN',
 'METHYLMALONATE',
 'MMV688553',
 'MMV688179',
 'P8S',
 'C44H79N1O10P1',
 'ANASTROZOLE',
 'C65H110N1O6',
 'C53H106N1O6',
 'C46H91N1O8P1',
 'MALEATE',
 'PEP',
 'C42H76O10P1',
 'MMV652003',
 'C43H88N1O5',
 'DEET',
 'SUBERATE',
 'C45H82N1O12S1',
 'C75H137O17P2',
 'NORLEUCINE',
 'LEUCINE',
 'TRYPTAMINE',
 'C41H72O13P1',
 'C42H70O10P1',
 'C43H84N1O6',
 'C49H90N1O6',
 'C43H79N1O8P1',
 'C73H133O17P2',
 'TETRACONAZOLE',
 'HEMIHYDRATE',
 'CAFFEINE',
 'C22H43N1O7P1',
 'TRIFLUMURON',
 'BC',
 'C37H68O8P1',
 'DBP',
 'PROLINE',
 'C55H104N1O6',
 'GLUCONATE',
 'C47H91N1O8P1'

## Clean (and extend) metadata

### 1) Harmization
+ Here, undefiend entries will be harmonized (instead of having a huge variation of None,"", "N/A" etc.)
+ The ``repair_inchi_inchikey_smiles`` function will correct misplaced metadata (e.g. inchikeys entered as inchi etc.) and harmonize the entry strings.

In [8]:
from tqdm.notebook import tqdm
from matchms.filtering import harmonize_undefined_inchikey, harmonize_undefined_inchi, harmonize_undefined_smiles
from matchms.filtering import repair_inchi_inchikey_smiles

def clean_metadata(s):
    s = harmonize_undefined_inchikey(s)
    s = harmonize_undefined_inchi(s)
    s = harmonize_undefined_smiles(s)
    s = repair_inchi_inchikey_smiles(s)
    return s

spectrums = [clean_metadata(s) for s in tqdm(spectrums)]

In [9]:
count_annotations(spectrums)

Inchis: 138059 -- 14901 unique
Smiles: 100075 -- 23061 unique
Inchikeys: 452 -- 303 unique (first 14 characters)


### 2) Convert entries where possible
Where possible (and necessary, i.e. missing): Convert between smiles, inchi, inchikey to complete metadata. This is done using functions from rdkit.

In [10]:
from tqdm.notebook import tqdm
from matchms.filtering import derive_inchi_from_smiles, derive_smiles_from_inchi
from matchms.filtering import derive_inchikey_from_inchi

def clean_metadata2(s):
    s = derive_inchi_from_smiles(s)
    s = derive_smiles_from_inchi(s)
    s = derive_inchikey_from_inchi(s)
    return s

spectrums = [clean_metadata2(s) for s in tqdm(spectrums)]

  0%|          | 0/210407 [00:00<?, ?it/s]

In [11]:
count_annotations(spectrums)

Inchis: 149639 -- 19520 unique
Smiles: 149232 -- 27401 unique
Inchikeys: 149202 -- 16427 unique (first 14 characters)


In [12]:
pickle.dump(spectrums, 
            open(os.path.join(path_data, 'GNPS_all', 'ALL_GNPS_cleaned_by_matchms.pickle'), "wb"))

In [3]:
import pickle
outfile = os.path.join(path_data, 'GNPS_all', 'ALL_GNPS_cleaned_by_matchms.pickle')
with open(outfile, 'rb') as file:
    spectrums = pickle.load(file)

In [4]:
count_annotations(spectrums)

Inchis: 149639 -- 19520 unique
Smiles: 149232 -- 27401 unique
Inchikeys: 149202 -- 16427 unique (first 14 characters)


## Run spectra with missing SMILES/InChI against PubChem

- Using function from `matchmsextras` which can be installed with `pip install matchmsextras`

In [ ]:
final_names = []
for spectrum in tqdm(spectrums):
    name_original = spectrum.get("compound_name")
    name = name_original.replace("F dial M", "")
    # Remove last word if likely not correct:
    if name.split(" ")[-1] in ["M", "M?", "?", "M+2H/2", "MS34+Na", "M]", "Cat+M]", "Unk", "--"]:
        name = " ".join(name.split(" ")[:-1]).strip()
    if name != name_original:
        final_names.append((name_original, name))
        print(f"Changed compound name from {name_original} to {name}.")
        spectrum.set("compound_name", name)

In [ ]:
from matchmsextras.pubchem_lookup import lookup_metadata_completion

spectrums_lookup = []
for i, s in enumerate(tqdm(spectrums)):
    spectrums_lookup.append(lookup_metadata_completion(s, search_depth=10))

In [4]:
count_annotations(spectrums)

Inchis: 184755 -- 22494 unique
Smiles: 184718 -- 29919 unique
Inchikeys: 184698 -- 17738 unique (first 14 characters)


In [ ]:
pickle.dump(spectrums_lookup, 
            open(os.path.join(path_data, 'GNPS_all', 'ALL_GNPS_matchms_manual_pubchem.pickle'), "wb"))

### To import the cleaned data:

In [2]:
import pickle
outfile = os.path.join(path_data, 'GNPS_all', 'ALL_GNPS_matchms_manual_pubchem.pickle')
with open(outfile, 'rb') as file:
    spectrums = pickle.load(file)

In [7]:
#from matchms.exporting import save_as_msp
filename = os.path.join(path_data, "GNPS_all",
                        "ALL_GNPS_210125_matchms_pubchem_cleaned.msp")  
save_as_msp(spectrums, filename)

In [6]:
len(spectrums), spectrums[0].get("ionmode")

(210407, 'positive')

In [27]:
for spec in spectrums:
    if spec.get("adduct") in ['[M+CH3COO]-/[M-CH3]-',
                             '[M-H]-/[M-Ser]-',
                             '[M-CH3]-']:
        if spec.get("ionmode") != "negative":
            spec.set("ionmode", "negative")

In [28]:
spectrums_positive = []
spectrums_negative = []
for i, spec in enumerate(spectrums):
    if spec.get("ionmode") == "positive":
        spectrums_positive.append(spec)
    elif spec.get("ionmode") == "negative":
        spectrums_negative.append(spec)
    else:
        print(f"No ionmode found for spectrum {i} ({spec.get('ionmode')})")

No ionmode found for spectrum 152657 (n/a)
No ionmode found for spectrum 152712 (n/a)
No ionmode found for spectrum 156170 (n/a)
No ionmode found for spectrum 156189 (n/a)
No ionmode found for spectrum 158895 (n/a)
No ionmode found for spectrum 158896 (n/a)
No ionmode found for spectrum 160921 (n/a)


In [34]:
len(spectrums_negative), len(spectrums_positive)

(65709, 144691)

In [35]:
len(spectrums) - (len(spectrums_negative) + len(spectrums_positive))

7

In [38]:
pickle.dump(spectrums_negative, 
            open(os.path.join(path_data, 'GNPS_all', 'ALL_GNPS_210125_negative_cleaned.pickle'), "wb"))

In [43]:
pickle.dump(spectrums_positive, 
            open(os.path.join(path_data, 'GNPS_all', 'ALL_GNPS_210125_positive_cleaned.pickle'), "wb"))

In [41]:
count_annotations(spectrums_positive) 

Inchis: 123908 -- 19500 unique
Smiles: 123885 -- 24606 unique
Inchikeys: 123866 -- 15489 unique (first 14 characters)


In [42]:
len(spectrums_positive)

144691

## Many spectra contain very few peaks!
- We here remove spectra with less than 5 peaks with m/z values in the range between 10.0 and 1000.0 Da
- We then make another subselection of only spectra which are fully annotated (InChIKey + SMILES/InChI)

In [44]:
number_of_peaks = np.array([len(s.peaks) for s in spectrums_positive])

In [52]:
print(f"{np.sum(number_of_peaks < 5)} spectra have < 5 peaks")

15215 spectra have < 5 peaks


In [55]:
from matchms.filtering import select_by_mz
from matchms.filtering import normalize_intensities
from matchms.filtering import require_minimum_number_of_peaks

def minimal_processing(spectrum):
    spectrum = normalize_intensities(spectrum)
    spectrum = select_by_mz(spectrum, mz_from=10.0, mz_to=1000.0)
    spectrum = require_minimum_number_of_peaks(spectrum, n_required=5)
    return spectrum

In [59]:
spectrums_pos_processing = [minimal_processing(s) for s in spectrums_positive]
spectrums_pos_processing = [s for s in spectrums_pos_processing if s is not None]
count_annotations(spectrums_pos_processing)

Inchis: 109775 -- 18686 unique
Smiles: 109751 -- 23726 unique
Inchikeys: 109739 -- 15062 unique (first 14 characters)


In [61]:
pickle.dump(spectrums_pos_processing, 
            open(os.path.join(path_data, 'GNPS_all', 'ALL_GNPS_210125_positive_processed.pickle'), "wb"))
len(spectrums_pos_processing)

129411

## Creating a subselection with only fully annotated spectra
- all annotated with InChIKey + SMILES and/or InChI
- all with >= 5 peaks with m/z between 10.0 and 1000.0 Da
- all positive ionization mode

In [66]:
spectrums_pos_annotated = []
for spec in spectrums_pos_processing:
    inchikey = spec.get("inchikey")
    if inchikey is not None and len(inchikey)>13:
        if spec.get("smiles") or spec.get("inchi"):
            spectrums_pos_annotated.append(spec)

len(spectrums_pos_annotated)

109734

In [67]:
count_annotations(spectrums_pos_annotated)

Inchis: 109734 -- 18671 unique
Smiles: 109734 -- 23710 unique
Inchikeys: 109734 -- 15062 unique (first 14 characters)


In [ ]:
pickle.dump(spectrums_pos_processing, 
            open(os.path.join(path_data, 'GNPS_all', 'ALL_GNPS_210125_positive_processed_annotated.pickle'), "wb"))
len(spectrums_pos_processing)